In [5]:
import requests
import json
import pandas as pd

def get_markets(api_key, secret, passphrase):
    url = "https://clob.polymarket.com/markets"
    headers = {
        'Authorization': f'Bearer {api_key}',
        # Add HMAC signature logic here
    }
    response = requests.get(url, headers=headers)
    return response.json()

In [11]:
import pandas as pd
from IPython.display import display

markets = get_markets('', '', '')

In [12]:

df = pd.DataFrame(markets)
df.to_csv('market_analysis.csv')
display(df)

,data,next_cursor,limit,count
0,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
1,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
2,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
3,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
4,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
...,...,...,...,...
995,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
996,"{'enable_order_book': False, 'active': False, ...",MTAwMA==,1000,1000
997,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000
998,"{'enable_order_book': False, 'active': True, '...",MTAwMA==,1000,1000


In [13]:
# %% [markdown]
# # Fetch all Polymarket markets into a DataFrame

# %% [code]
import requests
import pandas as pd
import time

def fetch_markets(limit=1000, offset=0, closed=None):
    """
    Fetch markets from Polymarket Gamma API.
    Returns list of market dicts.
    """
    base_url = "https://gamma-api.polymarket.com/markets"
    params = {
        "limit": limit,
        "offset": offset
    }
    if closed is not None:
        params["closed"] = str(closed).lower()
    resp = requests.get(base_url, params=params)
    resp.raise_for_status()
    return resp.json()

# %% [code]
# Example: fetch first page
markets_page = fetch_markets(limit=1000, offset=0, closed=False)
len(markets_page)

# %% [code]
# Loop through pages to build full list (or until some maximum)
all_markets = []
limit = 1000
offset = 0
while True:
    print(f"Fetching offset {offset}")
    page = fetch_markets(limit=limit, offset=offset, closed=False)
    if not page:
        break
    all_markets.extend(page)
    # if fewer than limit returned, no more pages
    if len(page) < limit:
        break
    offset += limit
    # be polite with API
    time.sleep(0.2)

# %% [code]
# Create DataFrame
df = pd.DataFrame(all_markets)

# %% [code]
# Select a subset of useful columns, rename if you like
cols = [
    "id",
    "question",
    "slug",
    "conditionId",
    "category",
    "startDate",
    "endDate",
    "volumeNum",
    "liquidityNum",
    "active",
    "closed"
]
# Some fields may be missing — use .get or fillna
df_sel = df.reindex(columns=cols)
df_sel = df_sel.rename(columns={
    "volumeNum": "volume",
    "liquidityNum": "liquidity"
})

# %% [code]
# Display sorted by volume descending
df_sel = df_sel.sort_values(by=["volume"], ascending=False)
df_sel.head(20)

# %% [markdown]
# ## Further possibilities
# - You can filter by category (e.g., df_sel[df_sel["category"]=="Politics"]).
# - Convert date columns to pandas datetime: `pd.to_datetime(df_sel["startDate"])`.
# - Export to CSV: `df_sel.to_csv("polymarket_markets.csv", index=False)`.
# - For live updates you could re-run the fetch periodically, or use the WebSocket endpoints.


Fetching offset 0


,id,question,slug,conditionId,category,startDate,endDate,volume,liquidity,active,closed
303,525362,Will George Russell be the 2025 Drivers Champion?,will-george-russell-be-the-2025-drivers-champion,0x7e8953cc66f9dabfe80fb0d1a9284bffb7a293f700b2...,NaN,2025-02-26T19:34:12.998Z,2025-12-07T12:00:00Z,2.613805e+07,520752.26286,True,False
10,516719,Russia x Ukraine ceasefire in 2025?,russia-x-ukraine-ceasefire-in-2025,0x8ee2f1640386310eb5e7ffa596ba9335f2d324e303d2...,NaN,2024-12-29T22:51:33.18Z,2025-12-31T12:00:00Z,2.456510e+07,417200.29870,True,False
189,520930,Jerome Powell out as Fed Chair in 2025?,will-trump-remove-jerome-powell,0x01d5f475cb30704216ae9906d369c0b2991cafec882a...,NaN,2025-01-29T23:13:22.244Z,2025-12-31T12:00:00Z,1.061820e+07,69097.07761,True,False
98,517014,Will The Fantastic Four: First Steps be the to...,will-the-fantastic-four-first-steps-be-the-top...,0x1cfde498b977cf83dd6a0276fb287de98dbeb79554c6...,NaN,2025-01-02T18:47:24.03753Z,2025-12-31T12:00:00Z,1.038551e+07,429232.61207,True,False
2,516710,US recession in 2025?,us-recession-in-2025,0xfa48a99317daef1654d5b03e30557c4222f276657275...,NaN,2025-01-08T01:33:54.924Z,2026-02-28T12:00:00Z,9.875135e+06,92560.96030,True,False
102,517018,Will Thunderbolts be the top grossing movie of...,will-thunderbolts-be-the-top-grossing-movie-of...,0x46fb01da8662e5eb2496b600bbe874d230f05f3e99b3...,NaN,2025-01-02T18:49:13.002797Z,2025-12-31T12:00:00Z,9.729243e+06,399519.22181,True,False
100,517016,Will Captain America: Brave New World be the t...,will-captain-america-new-world-order-be-the-to...,0x2993e8c18922f93787756e02dc262c193b79f05c7b95...,NaN,2025-01-02T18:47:42.296Z,2025-12-31T12:00:00Z,9.296278e+06,449683.18283,True,False
103,517019,Will How to Train Your Dragon be the top gross...,will-how-to-train-your-dragon-be-the-top-gross...,0x22fa7126aad142e3e7cd7dc6ae6669b86e28ff8ae6f0...,NaN,2025-01-02T18:50:19.06847Z,2025-12-31T12:00:00Z,8.077479e+06,464675.59581,True,False
207,521918,Will UCR hold the most seats in the Chamber of...,will-ucr-hold-the-most-seats-in-the-chamber-of...,0x937d7afc09f1d2a16a4daa1928798725ba97c809bcca...,NaN,2025-02-06T00:03:20.627Z,2025-10-26T12:00:00Z,8.073938e+06,30135.55975,True,False
8,516716,Khamenei out as Supreme Leader of Iran in 2025?,khamenei-out-as-supreme-leader-of-iran-in-2025,0x1b6f76e5b8587ee896c35847e12d11e75290a8c3934c...,NaN,2024-12-29T22:39:48.361Z,2025-12-31T12:00:00Z,7.781709e+06,82592.85720,True,False
